# Scenario Generator

This jupyter based interface has been designed to support scenario runs for the Ethiopia PATHWAYS project. 

The interface is built upon [OnSSET](http://www.onsset.org/) developed by KTH division of Energy Systems to provide an easy and quick way to generate electrification investment scenarios for the Ethiopia PATHWAYS project. 

#### Start by importing the code 

In [22]:
from onsset import *
from IPython.display import display, Markdown, HTML
%matplotlib inline

2021-08-30 10:02:45,652		Loaded backend module://ipykernel.pylab.backend_inline version unknown.


# 1. GIS data selection

First, run the cell below to browse to the directory where your calibrated input CSV file is located at and select the input file.


In [23]:
import tkinter as tk
from tkinter import filedialog, messagebox
from openpyxl import load_workbook
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)
messagebox.showinfo('OnSSET', 'Open the input file with extracted GIS data')
input_file = filedialog.askopenfilename()

onsseter = SettlementProcessor(input_file)
onsseter.df['IsUrban'] = 0
onsseter.df['Conflict'] = 0
onsseter.df['PerCapitaDemand'] = 0

# 2. Modelling period and target electrification rate

Next, define the modelling period and the electrification rate to be achieved by the years of the analysis.

In [24]:
start_year = 2018
end_year = 2070
yearsofanalysis = [2025, 2030, 2040, 2050, 2060, 2070]
time_steps = {2025: 7, 
              2030: 5, 
              2040: 10, 
              2050: 10, 
              2060: 10, 
              2070: 10}

# Electrification rate target. E.g. 1 = 100%, 0.5 = 50%
eleclimits = {2025: 1, 
              2030: 1, 
              2040: 1, 
              2050: 1, 
              2060: 1, 
              2070: 1}

time_step_number = {2025: 1, 
                    2030: 2, 
                    2040: 3, 
                    2050: 4, 
                    2060: 5, 
                    2070: 6}

In [25]:
# This is the maximum generation capacity (in MW) that can be added to the grid in one year. 
# "999999999" means that it is effectively unlimited.
annual_grid_cap_gen_limit = {2025: 999999999, 
                             2030: 999999999, 
                             2040: 999999999, 
                             2050: 999999999, 
                             2060: 999999999, 
                             2070: 999999999}       

In [26]:
# Buffer distance (km) from the current grid network for automatic connection to the grid. 
# 5 km reflects approximately 65% of the population, 25 km reflects approximately 95% of the population. 
# 0 means that there is no automatic connection to the grid, each settlement will consider both on- and off-grid alternatives
auto_intensification = {2025: 5, 
                        2030: 25, 
                        2040: 0, 
                        2050: 0, 
                        2060: 0, 
                        2070: 0}

In [27]:
# This is the maximum amount of new households that can be connected to the grid in one year (thousands).
# "999999999" means that it is effectively unlimited.
annual_new_grid_connections_limit = {2025: 0, 
                                     2030: 0, 
                                     2040: 999999999, 
                                     2050: 999999999, 
                                     2060: 999999999, 
                                     2070: 999999999} 

### b. Technology specifications & costs

The cell below contains all the information that is used to calculate the levelized costs for all the technologies, including grid. These default values should be updated to reflect the most accurate values in the country. There are currently 7 potential technologies to include in the model:
* Grid
* PV Mini-grid
* Wind Mini-grid
* Hydro Mini-grid
* Diesel Mini-grid
* PV Stand-alone systems
* Diesel Stand-alone systems

First, decide whether to include diesel technologies or not:

In [28]:
diesel_techs = 0                      ### 0 = diesel NOT included, 1 = diesel included 

The grid generation cost and grid power plants capital cost below for each time-step is given from the OSeMOSYS analysis

In [29]:
### This is the grid cost electricity USD/kWh as expected in the years of the analysis
grid_generation_cost = {2025: 0.06, 
                        2030: 0.08, 
                        2040: 0.09, 
                        2050: 0.09, 
                        2060: 0.10, 
                        2070: 0.10}

### The cost in USD/kW to for power generation capacity additions to the grid
grid_power_plants_capital_cost = {2025: 3338, 
                                  2030: 3354, 
                                  2040: 2664, 
                                  2050: 2222, 
                                  2060: 2313, 
                                  2070: 2447}

In [30]:
### The fraction of electricity lost in transmission and distribution (percentage) 
grid_losses = {2025: 0.122, 
               2030: 0.122, 
               2040: 0.122, 
               2050: 0.122, 
               2060: 0.122, 
               2070: 0.122}                     

In [31]:
### This is the diesel price in USD/liter as expected in the years of the analysis.
diesel_price = {2025: 0.85, 
                2030: 0.85*1.44, 
                2040: 0.85*1.49, 
                2050: 0.85*1.74, 
                2060: 0.85*1.77, 
                2070: 0.85*1.79}                   

In [32]:
### Stand-alone Diesel capital cost (USD/kW) as expected in the years of the analysis
sa_diesel_capital_cost = {2025: 999999, 
                          2030: 999999, 
                          2040: 999999, 
                          2050: 999999, 
                          2060: 999999, 
                          2070: 999999}

### Mini-grid Diesel capital cost (USD/kW) as expected in the years of the analysis
mg_diesel_capital_cost = {2025: 721, 
                          2030: 721, 
                          2040: 721, 
                          2050: 721, 
                          2060: 721, 
                          2070: 721}

### Mini-grid PV capital cost (USD/kW) as expected in the years of the analysis
mg_pv_capital_cost = {2025: 2829, 
                      2030: 2540, 
                      2040: 2252, 
                      2050: 1977, 
                      2060: 1703, 
                      2070: 1428}

### Mini-grid Wind capital cost (USD/kW) as expected in the years of the analysis
mg_wind_capital_cost = {2025: 4356, 
                        2030: 4285, 
                        2040: 4213, 
                        2050: 4117, 
                        2060: 4021, 
                        2070: 3926}

### Mini-grid Hydro capital cost (USD/kW) as expected in the years of the analysis
mg_hydro_capital_cost = {2025: 3000, 
                         2030: 3000, 
                         2040: 3000, 
                         2050: 3000, 
                         2060: 3000, 
                         2070: 3000}          

In [33]:
### Stand-alone PV capital cost (USD/kW) for household systems under 20 W
sa_pv_capital_cost_1 = {2025: 9067, 
                        2030: 8143, 
                        2040: 7218, 
                        2050: 6338, 
                        2060: 5458, 
                        2070: 4577}
### Stand-alone PV capital cost (USD/kW) for household systems between 21-50 W
sa_pv_capital_cost_2 = {2025: 8487, 
                        2030: 7621, 
                        2040: 6756, 
                        2050: 5932, 
                        2060: 5108, 
                        2070: 4285}
### Stand-alone PV capital cost (USD/kW) for household systems between 51-100 W
sa_pv_capital_cost_3 = {2025: 6165, 
                        2030: 5537, 
                        2040: 4908, 
                        2050: 4310, 
                        2060: 3711, 
                        2070: 3113}
### Stand-alone PV capital cost (USD/kW) for household systems between 101-1000 W
sa_pv_capital_cost_4 = {2025: 4316, 
                        2030: 3876, 
                        2040: 3436, 
                        2050: 3017, 
                        2060: 2598, 
                        2070: 2179}
### Stand-alone PV capital cost (USD/kW) for household systems over 1 kW
sa_pv_capital_cost_5 = {2025: 6710, 
                        2030: 6026, 
                        2040: 5342, 
                        2050: 4690, 
                        2060: 4039, 
                        2070: 3387}           

The cells below contain additional technology specifications

In [34]:
discount_rate = 0.10                  ### E.g. 0.10 means a discount rate of 10%

# Transmission and distribution costs
existing_grid_cost_ratio = 0.1        ### The additional cost per round of electrification (percentage)
hv_line_capacity=69 # kV
hv_line_cost=53000 # USD/km
mv_line_cost = 7000 # USD/kW
mv_line_capacity=50 # kV
mv_line_max_length=50 # km
mv_increase_rate=0.1
max_mv_line_dist = 100 # km
MV_line_amperage_limit = 8  # Ampere (A)
lv_line_capacity=0.24 #kV
lv_line_max_length=0.8 # km
lv_line_cost=4250 # USD/km
service_Transf_type=50  # kVa
service_Transf_cost=4250  # $/unit
max_nodes_per_serv_trans=300  # maximum number of nodes served by each service transformer
hv_lv_transformer_cost=25000 # USD/unit
hv_mv_transformer_cost=25000 # USD/unit
mv_lv_transformer_cost=10000 # USD/unit
mv_mv_transformer_cost=10000 # USD/unit

Technology.set_default_values(base_year=start_year, start_year=start_year, end_year=end_year, discount_rate=discount_rate,
                             HV_line_type=hv_line_capacity, HV_line_cost=hv_line_cost, MV_line_type=mv_line_capacity,
                             MV_line_amperage_limit=MV_line_amperage_limit, MV_line_cost=mv_line_cost, LV_line_type=lv_line_capacity,
                             LV_line_cost=lv_line_cost, LV_line_max_length=lv_line_max_length, 
                             service_Transf_type=service_Transf_type, service_Transf_cost = service_Transf_cost,
                             max_nodes_per_serv_trans=max_nodes_per_serv_trans, MV_LV_sub_station_cost=mv_lv_transformer_cost,
                             MV_MV_sub_station_cost=mv_mv_transformer_cost, HV_LV_sub_station_cost=hv_lv_transformer_cost,
                             HV_MV_sub_station_cost=hv_mv_transformer_cost)


# 7. Start a scenario run, which calculate and compare technology costs for every settlement in the country

Based on the previous calculation this piece of code identifies the LCoE that every off-grid technology can provide, for each single populated settlement of the selected country. The cell then takes all the currently grid-connected points in the country, and looks at the points within a certain distance from them, to see if it is more economical to connect them to the grid, or to use one of the off-grid technologies calculated above. Once more points are connected to the grid, the process is repeated, so that new points close to those points might also be connected. This is repeated until there are no new points to connect to the grid.

In [35]:
onsseter.current_mv_line_dist(2018)

for year in yearsofanalysis:
    end_year_pop = 1
    prioritization = 2
        
    eleclimit = eleclimits[year]
    time_step = time_steps[year]
    time_step_no = time_step_number[year]
    
    # Centralized grid costs
    grid_calc = Technology(om_of_td_lines=0.02,
                        distribution_losses=grid_losses[year],
                        connection_cost_per_hh=125,
                        base_to_peak_load_ratio=0.8,
                        capacity_factor=1,
                        tech_life=30,
                        grid_capacity_investment=grid_power_plants_capital_cost[year],
                        grid_price=grid_generation_cost[year])

    # Mini-grid hydro costs
    mg_hydro_calc = Technology(om_of_td_lines=0.02,
                            distribution_losses=0.05,
                            connection_cost_per_hh=100,
                            base_to_peak_load_ratio=0.85,
                            capacity_factor=0.5,
                            tech_life=30,
                            capital_cost=mg_hydro_capital_cost[year],
                            om_costs=0.03,
                            )

    # Mini-grid wind costs
    mg_wind_calc = Technology(om_of_td_lines=0.02,
                            distribution_losses=0.05,
                            connection_cost_per_hh=100,
                            base_to_peak_load_ratio=0.85,
                            capital_cost=mg_wind_capital_cost[year],
                            om_costs=0.02,
                            tech_life=20,
                            )

    # Mini-grid PV costs
    mg_pv_calc = Technology(om_of_td_lines=0.02,
                        distribution_losses=0.05,
                        connection_cost_per_hh=100,
                        base_to_peak_load_ratio=0.85,
                        tech_life=20,
                        om_costs=0.015,
                        capital_cost=mg_pv_capital_cost[year]                        
                       )

    # Stand-alone PV costs
    sa_pv_calc = Technology(base_to_peak_load_ratio=0.9,
                        tech_life=15,
                        om_costs=0.02,
                        capital_cost={0.020: sa_pv_capital_cost_1[year], 
                                      0.050: sa_pv_capital_cost_2[year], 
                                      0.100: sa_pv_capital_cost_3[year], 
                                      1: sa_pv_capital_cost_4[year], 
                                      5: sa_pv_capital_cost_5[year]},
                        standalone=True
                        )

    # Mini-grid diesel costs
    mg_diesel_calc = Technology(om_of_td_lines=0.02,
                            distribution_losses=0.05,
                            connection_cost_per_hh=100,
                            base_to_peak_load_ratio=0.85,
                            capacity_factor=0.7,
                            tech_life=15,
                            om_costs=0.1,
                            capital_cost=mg_diesel_capital_cost[year]
                            )

    # Stand-alone diesel costs
    sa_diesel_calc = Technology(base_to_peak_load_ratio=0.9,
                            capacity_factor=0.5,
                            tech_life=10,
                            om_costs=0.1,
                            capital_cost=sa_diesel_capital_cost[year],
                            standalone=True)
    
    sa_diesel_cost = {'diesel_price': diesel_price[year],
                          'efficiency': 0.28,
                          'diesel_truck_consumption': 14,
                          'diesel_truck_volume': 300}


    mg_diesel_cost = {'diesel_price':diesel_price[year],
                          'efficiency': 0.33,
                          'diesel_truck_consumption': 33.7,
                          'diesel_truck_volume': 15000}
    
    start_year = year - time_step
    end_year = year
    
    grid_cap_gen_limit = time_step * annual_grid_cap_gen_limit[year] * 1000
    grid_connect_limit = time_step * annual_new_grid_connections_limit[year] * 1000
    
    onsseter.set_scenario_variables(year, 1, 1, time_step,
                                            start_year, 6, 6, 1, 0, time_step_no)
    
    onsseter.diesel_cost_columns(sa_diesel_cost, mg_diesel_cost, year)

    onsseter.calculate_off_grid_lcoes(mg_hydro_calc, mg_wind_calc, mg_pv_calc, sa_pv_calc, mg_diesel_calc,
                                        sa_diesel_calc, year, start_year, end_year, time_step,
                                     diesel_techs=diesel_techs)

    onsseter.pre_electrification(grid_generation_cost[year], year, time_step, start_year)

    onsseter.df[SET_LCOE_GRID + "{}".format(year)], onsseter.df[SET_MIN_GRID_DIST + "{}".format(year)], onsseter.df[
                SET_ELEC_ORDER + "{}".format(year)], onsseter.df[SET_MV_CONNECT_DIST] = onsseter.elec_extension(grid_calc, 
                    max_mv_line_dist, year, start_year, end_year, time_step,
                    grid_cap_gen_limit, grid_connect_limit, auto_intensification=auto_intensification[year],
                    prioritization=prioritization)

    onsseter.results_columns(mg_hydro_calc, mg_wind_calc, mg_pv_calc, sa_pv_calc, mg_diesel_calc,
                                     sa_diesel_calc, grid_calc, year)

    onsseter.calculate_investments(mg_hydro_calc, mg_wind_calc, mg_pv_calc, sa_pv_calc, mg_diesel_calc,
                                           sa_diesel_calc, grid_calc, year, end_year, time_step)

    onsseter.apply_limitations(eleclimit, year, time_step, prioritization, auto_intensification[year])

    onsseter.final_decision(year)
    
    onsseter.calculate_new_capacity(mg_hydro_calc, mg_wind_calc, mg_pv_calc, sa_pv_calc, mg_diesel_calc,
                                            sa_diesel_calc, grid_calc, year, time_step)

    onsseter.pv_system_type(year, sa_pv_calc)
    
    onsseter.time_step_remaining_cap(mg_hydro_calc, mg_wind_calc, mg_pv_calc, sa_pv_calc, mg_diesel_calc,
                                    sa_diesel_calc, grid_calc, year, start_year, time_step_no)

2021-08-30 10:02:57,337		Determine current MV line length
2021-08-30 10:02:57,399		Calculate new connections
2021-08-30 10:02:57,463		Setting electrification demand as per target per year
2021-08-30 10:02:58,306		Calculate minigrid hydro LCOE
2021-08-30 10:02:59,333		315 potential hydropower sites were utilised to maximum capacity
2021-08-30 10:02:59,362		Calculate minigrid PV LCOE
2021-08-30 10:03:03,884		Calculate minigrid wind LCOE
2021-08-30 10:03:08,879		Calculate minigrid diesel LCOE
2021-08-30 10:03:13,163		Calculate standalone PV LCOE
2021-08-30 10:03:20,085		Determine minimum technology (off-grid)
2021-08-30 10:03:20,182		Determine minimum tech LCOE
2021-08-30 10:03:20,361		Define the initial electrification status
2021-08-30 10:03:24,295		Initially 1352 electrified
2021-08-30 10:03:26,950		Determine minimum overall
2021-08-30 10:03:27,038		Determine minimum overall LCOE
2021-08-30 10:03:27,131		Add technology codes
2021-08-30 10:03:27,212		Calculate investment cost
2021-08-30

The electrification rate achieved in 2025 is 0.0 %


2021-08-30 10:03:33,297		Calculate HV investment cost
2021-08-30 10:03:39,103		Calculate new connections
2021-08-30 10:03:39,151		Setting electrification demand as per target per year
2021-08-30 10:03:40,558		Calculate minigrid hydro LCOE
2021-08-30 10:03:41,668		59 potential hydropower sites were utilised to maximum capacity
2021-08-30 10:03:41,711		Calculate minigrid PV LCOE
2021-08-30 10:03:44,136		Calculate minigrid wind LCOE
2021-08-30 10:03:46,428		Calculate minigrid diesel LCOE
2021-08-30 10:03:48,769		Calculate standalone PV LCOE
2021-08-30 10:03:51,221		Determine minimum technology (off-grid)
2021-08-30 10:03:51,321		Determine minimum tech LCOE
2021-08-30 10:03:51,502		Define the initial electrification status
2021-08-30 10:03:53,379		Initially 18282 electrified
2021-08-30 10:03:59,635		Determine minimum overall
2021-08-30 10:03:59,754		Determine minimum overall LCOE
2021-08-30 10:03:59,844		Add technology codes
2021-08-30 10:03:59,944		Calculate investment cost
2021-08-30 10:

The electrification rate achieved in 2030 is 0.0 %


2021-08-30 10:04:03,887		Calculate MV investment cost
2021-08-30 10:04:07,099		Calculate HV investment cost
2021-08-30 10:04:11,474		Calculate new connections
2021-08-30 10:04:11,524		Setting electrification demand as per target per year
2021-08-30 10:04:12,889		Calculate minigrid hydro LCOE
C:\GitHub\onsset-Ethiopia\onsset\onsset.py:514: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(discounted_costs) / np.sum(discounted_generation)
2021-08-30 10:04:14,033		175 potential hydropower sites were utilised to maximum capacity
2021-08-30 10:04:14,082		Calculate minigrid PV LCOE
2021-08-30 10:04:15,087		Calculate minigrid wind LCOE
2021-08-30 10:04:15,968		Calculate minigrid diesel LCOE
2021-08-30 10:04:16,920		Calculate standalone PV LCOE
2021-08-30 10:04:17,969		Determine minimum technology (off-grid)
2021-08-30 10:04:18,065		Determine minimum tech LCOE
2021-08-30 10:04:18,278		Define the initial electrification status
2021-08-30 10:04:18,890		Initially 35792 e

...

2021-08-30 10:04:44,025		Calculate new capacity
2021-08-30 10:04:44,108		Calculate MV investment cost
2021-08-30 10:04:47,381		Calculate HV investment cost
2021-08-30 10:04:51,984		Calculate new connections
2021-08-30 10:04:52,055		Setting electrification demand as per target per year
2021-08-30 10:04:53,688		Calculate minigrid hydro LCOE
C:\GitHub\onsset-Ethiopia\onsset\onsset.py:514: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(discounted_costs) / np.sum(discounted_generation)
2021-08-30 10:04:54,875		1 potential hydropower sites were utilised to maximum capacity
2021-08-30 10:04:54,944		Calculate minigrid PV LCOE
2021-08-30 10:04:55,979		Calculate minigrid wind LCOE
2021-08-30 10:04:57,008		Calculate minigrid diesel LCOE
2021-08-30 10:04:58,101		Calculate standalone PV LCOE
2021-08-30 10:04:59,201		Determine minimum technology (off-grid)
2021-08-30 10:04:59,330		Determine minimum tech LCOE
2021-08-30 10:04:59,607		Define the initial electrification sta

The electrification rate achieved in 2060 is 0.0 %


2021-08-30 10:05:05,988		Calculate new capacity
2021-08-30 10:05:06,081		Calculate MV investment cost
2021-08-30 10:05:09,471		Calculate HV investment cost
2021-08-30 10:05:14,239		Calculate new connections
2021-08-30 10:05:14,317		Setting electrification demand as per target per year
2021-08-30 10:05:16,161		Calculate minigrid hydro LCOE
C:\GitHub\onsset-Ethiopia\onsset\onsset.py:514: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(discounted_costs) / np.sum(discounted_generation)
2021-08-30 10:05:17,487		1 potential hydropower sites were utilised to maximum capacity
2021-08-30 10:05:17,564		Calculate minigrid PV LCOE
2021-08-30 10:05:18,646		Calculate minigrid wind LCOE
2021-08-30 10:05:19,798		Calculate minigrid diesel LCOE
2021-08-30 10:05:20,930		Calculate standalone PV LCOE
2021-08-30 10:05:22,025		Determine minimum technology (off-grid)
2021-08-30 10:05:22,165		Determine minimum tech LCOE
2021-08-30 10:05:22,439		Define the initial electrification sta

The electrification rate achieved in 2070 is 0.0 %


2021-08-30 10:05:27,956		Calculate new capacity
2021-08-30 10:05:28,055		Calculate MV investment cost
2021-08-30 10:05:31,408		Calculate HV investment cost


# 8. Results, Summaries and Visualization
With all the calculations and grid-extensions complete, this block gets the final results on which technology was chosen for each point, how much capacity needs to be installed and what it will cost. Then the summaries, plots and maps are generated.

In [36]:
elements = []
for year in yearsofanalysis:
    elements.append("Population{}".format(year))
    elements.append("NewConnections{}".format(year))
    elements.append("Capacity{}".format(year))
    elements.append("Investment{}".format(year))

techs = ["Grid", "SA_Diesel", "SA_PV", "MG_Diesel", "MG_PV", "MG_Wind", "MG_Hydro"]

sumtechs = []
for year in yearsofanalysis:
    sumtechs.extend(["Population{}".format(year) + t for t in techs])
    sumtechs.extend(["NewConnections{}".format(year) + t for t in techs])
    sumtechs.extend(["Capacity{}".format(year) + t for t in techs])
    sumtechs.extend(["Investment{}".format(year) + t for t in techs])

summary = pd.Series(index=sumtechs, name='country')

for year in yearsofanalysis:
    for t in techs:
        summary.loc["Population{}".format(year) + t] = onsseter.df.loc[(onsseter.df[SET_MIN_OVERALL + '{}'.format(year)] == t + '{}'.format(year)) & (onsseter.df[SET_ELEC_FINAL_CODE + '{}'.format(year)] < 99), SET_POP + '{}'.format(year)].sum()
        summary.loc["NewConnections{}".format(year) + t] = onsseter.df.loc[(onsseter.df[SET_MIN_OVERALL + '{}'.format(year)] == t + '{}'.format(year)) & (onsseter.df[SET_ELEC_FINAL_CODE + '{}'.format(year)] < 99), SET_NEW_CONNECTIONS + '{}'.format(year)].sum()
        summary.loc["Capacity{}".format(year) + t] = onsseter.df.loc[(onsseter.df[SET_MIN_OVERALL + '{}'.format(year)] == t + '{}'.format(year)) & (onsseter.df[SET_ELEC_FINAL_CODE + '{}'.format(year)] < 99), SET_NEW_CAPACITY + '{}'.format(year)].sum()/1000
        summary.loc["Investment{}".format(year) + t] = onsseter.df.loc[(onsseter.df[SET_MIN_OVERALL + '{}'.format(year)] == t + '{}'.format(year)) & (onsseter.df[SET_ELEC_FINAL_CODE + '{}'.format(year)] < 99), SET_INVESTMENT_COST + '{}'.format(year)].sum()
        
index = techs + ['Total']
columns = []
for year in yearsofanalysis:
    columns.append("Population{}".format(year))
    columns.append("NewConnections{}".format(year))
    columns.append("Capacity{} (MW)".format(year))
    columns.append("Investment{} (million USD)".format(year))
                                                                                                                                           
summary_table = pd.DataFrame(index=index, columns=columns)
turn = 0
for years in yearsofanalysis:
    summary_table[columns[turn * 4]] = summary.iloc[turn * 28:turn * 28 + 7].astype(int).tolist() + [int(summary.iloc[turn * 28:turn * 28 + 7].sum())]
    summary_table[columns[turn * 4 + 1]] = summary.iloc[turn * 28 + 7:turn * 28 + 14].astype(int).tolist() + [int(summary.iloc[turn * 28 + 7:turn * 28 + 14].sum())]
    summary_table[columns[turn * 4 + 2]] = summary.iloc[turn * 28 + 14:turn * 28 + 21].astype(int).tolist() + [int(summary.iloc[turn * 28 + 14:turn * 28 + 21].sum())]
    summary_table[columns[turn * 4 + 3]] = [round(x/1e4)/1e2 for x in summary.iloc[turn * 28 + 21:turn * 28 + 28].astype(float).tolist()] + [round(summary.iloc[turn * 28 + 21:turn * 28 + 28].sum()/1e4)/1e2]
    turn += 1


<ipython-input-36-fd3aa4f5900e>:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  summary = pd.Series(index=sumtechs, name='country')


In [37]:
display(Markdown('### Summary \n These are the summarized results for full electrification of the selected country by the final year'))
summary_table

### Summary 
 These are the summarized results for full electrification of the selected country by the final year

,Population2025,NewConnections2025,Capacity2025 (MW),Investment2025 (million USD),Population2030,NewConnections2030,Capacity2030 (MW),Investment2030 (million USD),Population2040,NewConnections2040,...,Capacity2050 (MW),Investment2050 (million USD),Population2060,NewConnections2060,Capacity2060 (MW),Investment2060 (million USD),Population2070,NewConnections2070,Capacity2070 (MW),Investment2070 (million USD)
Grid,86162457,47352726,957,6861.82,138120541,14578056,952,6829.18,172489364,30941348,...,7927,19551.29,234273146,29590929,6159,16470.95,258124349,31897878,11886,32147.82
SA_Diesel,0,0,0,0.00,0,0,0,0.00,0,0,...,0,0.00,0,0,0,0.00,0,0,0,0.00
SA_PV,43149146,41063051,69,442.67,9089563,558224,2,21.28,7025192,528138,...,29,115.14,365438,0,35,91.26,338086,0,35,77.49
MG_Diesel,0,0,0,0.00,0,0,0,0.00,0,0,...,0,0.00,0,0,0,0.00,0,0,0,0.00
MG_PV,4587163,4429932,408,1357.26,2236853,411910,148,434.74,1240851,93284,...,266,656.94,2426427,0,110,227.88,2609846,0,237,338.98
MG_Wind,0,0,0,0.00,0,0,0,0.00,0,0,...,0,0.00,0,0,0,0.00,0,0,0,0.00
MG_Hydro,310278,310278,3,46.03,328883,18604,0,3.68,630612,47408,...,66,230.09,664580,0,10,36.01,103844,0,0,2.16
Total,134209046,93155989,1439,8707.78,149775841,15566794,1104,7288.88,181386020,31610179,...,8289,20553.46,237729592,29590929,6315,16826.11,261176128,31897878,12160,32566.45


In [38]:
df_in = onsseter.df
df_in['PVType2018'] = 1
df_in['NewConnections2018'] = 0
df_in['NewCapacity2018'] = 0
df_in['InvestmentCost2018'] = 0
df_in['TransmissionInvestmentCost2018'] = 0
df_in['TotalEnergyPerCell2018'] = df_in['ElecPopCalib'] * df_in['ResidentialDemandTierCustom2018'] * df_in[SET_ELEC_CURRENT]
SET_TRANSMISSION_INV = 'TransmissionInvestmentCost'

elements = ["1.Population", "2.New_Connections", "3.Capacity", "4.Investment", "5. Demand", "6. Transmission summaries", "7. Distribution summaries", "8. Capacity factor"]
techs = ["Grid", "MG_Diesel", "MG_PV", "MG_Wind", "MG_Hydro", "SA_PV_1", "SA_PV_2", "SA_PV_3", "SA_PV_4", "SA_PV_5"]
years = [2018, 2025, 2030, 2040, 2050, 2060, 2070]
tech_codes = [1, 4, 5, 6, 7, 8, 9, 10, 11, 12]
tech_costs = {1: grid_power_plants_capital_cost, 4: mg_diesel_capital_cost, 5: mg_pv_capital_cost, 6: mg_wind_capital_cost, 
              7: mg_hydro_capital_cost, 8: sa_pv_capital_cost_1, 9: sa_pv_capital_cost_2, 10: sa_pv_capital_cost_3,
              11: sa_pv_capital_cost_4, 12: sa_pv_capital_cost_5}


for year in years:
    df_in.loc[(df_in[SET_ELEC_FINAL_CODE + '{}'.format(year)] == 3) & (df_in['PVType' + '{}'.format(year)] == 1), SET_ELEC_FINAL_CODE + '{}'.format(year)] = 8
    df_in.loc[(df_in[SET_ELEC_FINAL_CODE + '{}'.format(year)] == 3) & (df_in['PVType' + '{}'.format(year)] == 2), SET_ELEC_FINAL_CODE + '{}'.format(year)] = 9
    df_in.loc[(df_in[SET_ELEC_FINAL_CODE + '{}'.format(year)] == 3) & (df_in['PVType' + '{}'.format(year)] == 3), SET_ELEC_FINAL_CODE + '{}'.format(year)] = 10
    df_in.loc[(df_in[SET_ELEC_FINAL_CODE + '{}'.format(year)] == 3) & (df_in['PVType' + '{}'.format(year)] == 4), SET_ELEC_FINAL_CODE + '{}'.format(year)] = 11
    df_in.loc[(df_in[SET_ELEC_FINAL_CODE + '{}'.format(year)] == 3) & (df_in['PVType' + '{}'.format(year)] == 5), SET_ELEC_FINAL_CODE + '{}'.format(year)] = 12

df_electrified = df_in.loc[df_in[SET_ELEC_CURRENT] == 1]
df_unelectrified = df_in.loc[df_in[SET_ELEC_CURRENT] == 0]

sumtechs = []
for element in elements:
    for tech in techs:
        sumtechs.append(element + "_" + tech)

total_rows = len(sumtechs)

electrified_sumtechs = pd.DataFrame(columns=years)
unelectrified_sumtechs = pd.DataFrame(columns=years)
combined_sumtechs = pd.DataFrame(columns=years)

for row in range(0, total_rows):
    electrified_sumtechs.loc[sumtechs[row]] = 0
    unelectrified_sumtechs.loc[sumtechs[row]] = 0
    combined_sumtechs.loc[sumtechs[row]] = 0

i = 0

# Adding pop summary (Million ppl)
for code in tech_codes:
    for year in years:
        electrified_sumtechs[year][sumtechs[i]] = sum(df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_POP + "{}".format(year)])/1000000
        unelectrified_sumtechs[year][sumtechs[i]] = sum(df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_POP + "{}".format(year)])/1000000
        combined_sumtechs[year][sumtechs[i]] = sum(df_in.loc[df_in[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_POP + "{}".format(year)]) / 1000000
    i += 1

# Adding connections summary (Million ppl)
for code in tech_codes:
    for year in years:
        electrified_sumtechs[year][sumtechs[i]] = sum(df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_NEW_CONNECTIONS + "{}".format(year)])/1000000
        unelectrified_sumtechs[year][sumtechs[i]] = sum(df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_NEW_CONNECTIONS + "{}".format(year)])/1000000
    i += 1

# Adding capacity summaries (MW)
for code in tech_codes:
    for year in years:
        electrified_sumtechs[year][sumtechs[i]] = sum(df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_NEW_CAPACITY + "{}".format(year)])/1000
        unelectrified_sumtechs[year][sumtechs[i]] = sum(df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_NEW_CAPACITY + "{}".format(year)])/1000
    i += 1

# Adding investment summaries (Million USD)
for code in tech_codes:
    for year in years:
        electrified_sumtechs[year][sumtechs[i]] = sum(df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_INVESTMENT_COST + "{}".format(year)])/1000000
        unelectrified_sumtechs[year][sumtechs[i]] = sum(df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_INVESTMENT_COST + "{}".format(year)])/1000000
    i += 1

# Adding demand summaries (GWh)
for code in tech_codes:
    for year in years:
        electrified_sumtechs[year][sumtechs[i]] = sum(df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_TOTAL_ENERGY_PER_CELL + "{}".format(year)])/1000000
        unelectrified_sumtechs[year][sumtechs[i]] = sum(df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_TOTAL_ENERGY_PER_CELL + "{}".format(year)])/1000000
    i += 1

# Adding transmission summaries (Million USD)
for code in tech_codes:
    for year in years:
        if code == 1:
            electrified_sumtechs[year][sumtechs[i]] = sum(df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code, SET_TRANSMISSION_INV + "{}".format(year)]) / 1000000
            unelectrified_sumtechs[year][sumtechs[i]] = sum(df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code, SET_TRANSMISSION_INV + "{}".format(year)])/1000000
    i += 1

# Adding distribution summaries (Million USD)
for code in tech_codes:
    for year in years:
        if (electrified_sumtechs[year][sumtechs[i-40]] > 0) & (code < 8):
            cost_dict = tech_costs[code]
            inv_cost = cost_dict[year]
            electrified_sumtechs[year][sumtechs[i]] = (electrified_sumtechs[year][sumtechs[i-30]]*1000000 -
                                                       electrified_sumtechs[year][sumtechs[i-10]]*1000000 -
                                                       electrified_sumtechs[year][sumtechs[i-40]]*1000 * inv_cost)/1000000
        if (unelectrified_sumtechs[year][sumtechs[i-40]] > 0) & (code < 8):
            cost_dict = tech_costs[code]
            inv_cost = cost_dict[year]
            unelectrified_sumtechs[year][sumtechs[i]] = (unelectrified_sumtechs[year][sumtechs[i-30]]*1000000 -
                                                       unelectrified_sumtechs[year][sumtechs[i-10]]*1000000 -
                                                       unelectrified_sumtechs[year][sumtechs[i-40]]*1000 * inv_cost)/1000000 
    i += 1

for code in tech_codes:
    for year in years:
        if code == 5 or code > 7:
            if electrified_sumtechs[year][sumtechs[i-40]] > 0:
                electrified_sumtechs[year][sumtechs[i]] = df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_GHI].mean()/8760
            if unelectrified_sumtechs[year][sumtechs[i - 40]] > 0:
                unelectrified_sumtechs[year][sumtechs[i]] = df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_GHI].mean() / 8760
        elif code == 6:
            if electrified_sumtechs[year][sumtechs[i - 40]] > 0:
                electrified_sumtechs[year][sumtechs[i]] = df_electrified.loc[df_electrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_WINDCF].mean()
            if unelectrified_sumtechs[year][sumtechs[i - 40]] > 0:
                unelectrified_sumtechs[year][sumtechs[i]] = df_unelectrified.loc[df_unelectrified[SET_ELEC_FINAL_CODE + "{}".format(year)] == code][SET_WINDCF].mean()
    i += 1

all_sumtechs = electrified_sumtechs + unelectrified_sumtechs

## 9. Exporting results

This code generates five csv files:
 - one containing all the results for the scenario created
 - three containing the summaries for the scenario created
 - one containing some if the key input variables of the scenario

Before we proceed, please write the scenario_name in the first cell below. then move on to the next cell and run it to browse to the directory where you want to save your results. Sample file shall be located at .\ gep-onsset\sample_output. 

**Note that if you do not change the scenario name, the previous output files will be overwritten**

In [39]:
scenario_name = "New_Policies_Control_Run"

In [40]:
list1 = [('Start_year',start_year,'','',''), 
         ('End_year',yearsofanalysis,'','',''),
         ('End year electrification rate target',eleclimits,'','',''),
         ('Prioritization', prioritization,'','',''),
         ('Auto intensification distance', auto_intensification, '', '', 'Buffer distance (km) for automatic intensification if choosing prioritization 1'),
         ('discount_rate',discount_rate,'','',''),
         ('grid_generation_cost',grid_generation_cost,'','','This is the grid cost electricity USD/kWh as expected in the end year of the analysis'),
         ('grid_power_plants_capital_cost',grid_power_plants_capital_cost,'','','The cost in USD/kW to for capacity upgrades of the grid-connected power plants'),
         ('grid_losses',grid_losses,'','','The fraction of electricity lost in transmission and distribution (percentage)'),
         ('existing_grid_cost_ratio',existing_grid_cost_ratio,'','','The additional cost per round of electrification (percentage)'),
         ('diesel_price',diesel_price,'','','This is the diesel price in USD/liter as expected in the end year of the analysis'),
         ('sa_diesel_capital_cost',sa_diesel_capital_cost,'','','Stand-alone Diesel capital cost (USD/kW) as expected in the years of the analysis'),
         ('mg_diesel_capital_cost',mg_diesel_capital_cost,'','','Mini-grid Diesel capital cost (USD/kW) as expected in the years of the analysis'),
         ('mg_pv_capital_cost',mg_pv_capital_cost,'','','Mini-grid PV capital cost (USD/kW) as expected in the years of the analysis'),
         ('mg_wind_capital_cost',mg_wind_capital_cost,'','','Mini-grid Wind capital cost (USD/kW) as expected in the years of the analysis'),
         ('mg_hydro_capital_cost',mg_hydro_capital_cost,'','','Mini-grid Hydro capital cost (USD/kW) as expected in the years of the analysis'),
         ('sa_pv_capital_cost_1',sa_pv_capital_cost_1,'','','Stand-alone PV capital cost (USD/kW) for household systems under 20 W'),
         ('sa_pv_capital_cost_2',sa_pv_capital_cost_2,'','','Stand-alone PV capital cost (USD/kW) for household systems between 21-50 W'),
         ('sa_pv_capital_cost_3',sa_pv_capital_cost_3,'','','Stand-alone PV capital cost (USD/kW) for household systems between 51-100 W'),
         ('sa_pv_capital_cost_4',sa_pv_capital_cost_4,'','','Stand-alone PV capital cost (USD/kW) for household systems between 101-200 W'),
         ('sa_pv_capital_cost_5',sa_pv_capital_cost_5,'','','Stand-alone PV capital cost (USD/kW) for household systems over 200 W'),
         ('mv_line_cost',mv_line_cost,'','','Cost of MV lines in USD/km'),
         ('lv_line_cost',lv_line_cost,'','','Cost of LV lines in USD/km'),
         ('mv_line_capacity',mv_line_capacity,'','','Capacity of MV lines in kW/line'),
         ('lv_line_capacity',lv_line_capacity,'','','Capacity of LV lines in kW/line'),
         ('lv_line_max_length',lv_line_max_length,'','','Maximum length of LV lines (km)'),
         ('hv_line_cost',hv_line_cost,'','','Cost of HV lines in USD/km'),
         ('mv_line_max_length',mv_line_max_length,'','','Maximum length of MV lines (km)'),
         ('hv_lv_transformer_cost',hv_lv_transformer_cost,'','','Cost of HV/MV transformer (USD/unit)'),
         ('mv_increase_rate',mv_increase_rate,'','','percentage'),
         ('max_grid_extension_dist',max_mv_line_dist,'','','Maximum distance that the grid may be extended by means of MV lines'),
         ('annual_new_grid_connections_limit', annual_new_grid_connections_limit,'','','This is the maximum amount of new households that can be connected to the grid in one year (thousands)'),
         ('grid_capacity_limit',annual_grid_cap_gen_limit,'','','This is the maximum generation capacity that can be added to the grid in one year (MW)'),
         ]
labels = ['Variable','Value', 'Source', 'Comments', 'Description']
df_variables = pd.DataFrame.from_records(list1, columns=labels)

In [41]:
messagebox.showinfo('OnSSET', 'Browse to the folder where you want to save the outputs')

output_dir = filedialog.askdirectory()
output_dir_variables = os.path.join(output_dir, '{}_Variables.csv'.format(scenario_name))
output_dir_results = os.path.join(output_dir, '{}_Results.csv'.format(scenario_name))
output_dir_summaries = os.path.join(output_dir, '{}_Summaries.csv'.format(scenario_name))
output_dir_unelectrified_summaries = os.path.join(output_dir, '{}_Unelectrified_Summaries.csv'.format(scenario_name))
output_dir_electrified_summaries = os.path.join(output_dir, '{}_Electrified_Summaries.csv'.format(scenario_name))
output_dir_combined_summaries = os.path.join(output_dir, '{}_Combined_Summaries.csv'.format(scenario_name))

In [42]:
# Returning the result as a csv file
onsseter.df.to_csv(output_dir_results, index=False)

# Returning the input variables as a csv file
df_variables.to_csv(output_dir_variables, index=False)

# Returning the summaries for initially unelectrified settlements
unelectrified_sumtechs.to_csv(output_dir_unelectrified_summaries, index=True)

# Returning the summaries for initially electrified settlements
electrified_sumtechs.to_csv(output_dir_electrified_summaries, index=True)

# Returning the combined summaries for all settlements
all_sumtechs.to_csv(output_dir_combined_summaries, index=True)